In [3]:
import pandas as pd
import plotly.express as px
import numpy as np

In [3]:
df = pd.read_csv('../experiments/data.csv')
df['datetime'] = pd.to_datetime(df['datetime'])
df['time_from_start'] = df.groupby('experiment')['datetime'].transform(lambda x: (x - x.min()).dt.total_seconds() / 3600)


In [4]:

# Pivot the data: metrics become columns, keep experiment info
pivot_df = df.pivot_table(
    index=['datetime', 'time_from_start', 'experiment'], 
    columns='tag', 
    values='value'
).reset_index()

# Flatten column names
pivot_df.columns.name = None

In [5]:
# metric options
df['tag'].unique()

array(['Loss/train', 'Loss/val', 'Accuracy/train', 'Accuracy/val',
       'BLEU/val', 'LR'], dtype=object)

In [31]:
fig = px.line(pivot_df, 
              x='time_from_start', 
              y='Accuracy/val',
              color='experiment',
              markers=True,
              title='Loss Over Time')

fig.show()

### Creating Table

In [ ]:
import os
import json

def load_experiment_args(dir):
    args_file = os.path.join(dir, 'args.json')
    if os.path.exists(args_file):
        with open(args_file, 'r') as f:
            return json.load(f)
    return None


In [ ]:

# loop through each subdirectory in the experiments directory
experiments_dir = '../experiments'
experiment_args = {}
for subdir in os.listdir(experiments_dir):
    subdir_path = os.path.join(experiments_dir, subdir)
    if os.path.isdir(subdir_path):
        args = load_experiment_args(subdir_path)
        if args:
            experiment_args[subdir] = args

In [9]:
args_df = pd.DataFrame.from_dict(experiment_args, orient='index')
args_df

,data_dir,output_dir,encoder_d,decoder_d,embed_d,attention_d,num_layers,dropout,pretrained,epochs,batch_size,lr,weight_decay,grad_clip,accumulation_steps,patience,num_workers,resume,seed
exp1,crohme_images,experiments/exp1,256,256,256,256,2,0.50,False,100,32,0.0010,0.00010,5.0,1,10,4,None,42
exp2,crohme_images,experiments/exp2,64,64,64,64,1,0.50,False,200,128,0.0050,0.00010,5.0,1,10,8,None,42
exp3,crohme_images,experiments/exp3,512,512,512,512,3,0.45,True,200,32,0.0005,0.00005,1.0,1,20,6,None,42
